<a href="https://colab.research.google.com/github/immangwu/COPO/blob/main/Copy_of_response_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install nltk

In [ ]:
import nltk

nltk.download('all')

In [ ]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import gspread
import pandas as pd
from google.colab import auth

In [ ]:
from google.colab import auth
import gspread
from google.auth import default
#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
#defining my worksheet
worksheet = gc.open('Survey data').sheet1
#get_all_values gives a list of rows
rows = worksheet.get_all_values()
#Convert to a DataFrame
df = pd.DataFrame(rows)
df=df.iloc[:, 2]
df

In [ ]:
from textblob import TextBlob

def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity


In [ ]:
# create preprocess_text function
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    # Remove stop words
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    # Join the tokens back into a string
    processed_text = ' '.join(lemmatized_tokens)
    return processed_text
# apply the function df
df = df.apply(preprocess_text)
df

In [ ]:
# Create get_sentiment function to handle multiple texts
def get_sentiment(texts):
    sentiments = []
    for text in texts:
        scores = analyzer.polarity_scores(text)
        threshold = 0.1  # Adjust the threshold as needed
        sentiment = 1 if scores['pos'] - scores['neg'] >= threshold else 0
        sentiments.append(sentiment)
    return sentiments


In [ ]:
# initialize NLTK sentiment analyzer
analyzer = SentimentIntensityAnalyzer()
# create get_sentiment function
def get_sentiment(text):
    scores = analyzer.polarity_scores(text)
    sentiment = 1 if scores['pos'] > 0 else 0
    return sentiment
# apply get_sentiment function
i=[]
arr=[]
i = df.apply(get_sentiment)
arr = df.to_numpy()
arr



In [ ]:
positive=[];
negative=[];
for x in range(len(arr)):
  if i[x]>0:
    positive.append(arr[x]);
  else:
    negative.append(arr[x]);
positiveStr = ' '.join(map(str, positive))
negativeStr = ' '.join(map(str, negative))


In [43]:
import wordcloud
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display
#import fileupload
import io
import sys


def calculate_frequencies(file_contents,y):
    # Here is a list of punctuations and uninteresting words you can use to process your text
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    uninteresting_words = ["the", "a", "to", "if", "is", "it", "of", "and", "or", "an", "as", "i", "me", "my", \
    "we", "our", "ours", "you", "your", "yours", "he", "she", "him", "his", "her", "hers", "its", "they", "them", \
    "their", "what", "which", "who", "whom", "this", "that", "am", "are", "was", "were", "be", "been", "being", \
    "have", "has", "had", "do", "does", "did", "but", "at", "by", "with", "from", "here", "when", "where", "how", \
    "all", "any", "both","in", "each", "few", "more", "some", "such", "no", "nor", "too", "very", "can", "will", "just","on"]
    a=file_contents.split()
    new_list=[]
    for i in a:
        new=i.lower()
        if new not in punctuations and new not in uninteresting_words:
            new_list.append(new)
    result = {}
    for letter in new_list:
        if letter.isalpha():
            if letter not in result:
                result[letter]=0
            result[letter]+=1
    #return result

    # LEARNER CODE START HERE


    #wordcloud
    cloud = wordcloud.WordCloud(width=900,height=500, max_words=1628,relative_scaling=1,normalize_plurals=False )
    cloud.generate_from_frequencies(result)
    #storing
    images_dir = '/content/drive/My Drive/Images'
    if y==1:
      cloud.to_file(f"{images_dir}/Positive.jpg");
    else:
      cloud.to_file(f"{images_dir}/Negative.jpg");


    return cloud.to_array()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [44]:
myimage = calculate_frequencies(positiveStr,1)
myimage = calculate_frequencies(negativeStr,0)
